In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv(r"C:\Users\abhis\Downloads\adult.data", sep=',\s', na_values=["?"])
# df.dropna(inplace=True)

df=df.replace("<=50K",0).replace(">50K",1)
max=df.max(axis=0)
min= df.min(axis=0)
# mean= df.mean(axis=0)


# discretisation of age column
df['age'] = np.where(df['age'].between(16,25), 1, df['age'])
df['age'] = np.where(df['age'].between(26,35), 2, df['age'])
df['age'] = np.where(df['age'].between(36,45), 3, df['age'])
df['age'] = np.where(df['age'].between(46,55), 4, df['age'])
df['age'] = np.where(df['age'].between(56,65), 5, df['age'])
df['age'] = np.where(df['age'].between(66,75), 6, df['age'])
df['age'] = np.where(df['age'].between(76,85), 7, df['age'])
df['age'] = np.where(df['age'].between(86,95), 8, df['age'])

# discretisation of fnlwgt

df['fnlwgt']=pd.cut(x=df["fnlwgt"], bins= 5, labels=["f1","f2","f3","f4","f5"])

# discretisation of capital-gain ,capital-loss and hours-per-week

df["captial-gain"]=pd.cut(x=df['captial-gain'], bins=10, labels= ['cg1','cg2','cg3','cg4','cg5','cg6','cg7','cg8','cg9','cg10'])
df['capital-loss']=pd.cut(x=df['capital-loss'], bins=10, labels= ['cf1','cf2','cf3','cf4','cf5','cf6','cf7','cf8','cf9','cf10'])
df['hours-per-week']=pd.cut(x=df['hours-per-week'],bins=10, labels=['hw1','hw2','hw3','hw4','hw5','hw6','hw7','hw8','hw9','hw10'])


# filling null values with the most frequent one 
df.workclass.fillna("Private",inplace=True)
df["native country"].fillna("United-States",inplace=True)
df.occupation.fillna("Prof-specialty ",inplace=True)

# making test and training samples


df_m = df.sample(frac=2/3)
x_train= df_m.drop("income",axis=1)
x_train=df_m
y_train= df_m["income"]
x_test= df.drop(df_m.index)
y_test= x_test["income"]
x_test.drop("income",axis=1,inplace=True)
x_train= x_train.to_numpy()
y_train= y_train.to_numpy()
x_test=x_test.to_numpy()
y_test=y_test.to_numpy()

y_train



C:\Users\abhis\AppData\Local\Temp\ipykernel_28016\4219654605.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(r"C:\Users\abhis\Downloads\adult.data", sep=',\s', na_values=["?"])
C:\Users\abhis\AppData\Local\Temp\ipykernel_28016\4219654605.py:8: FutureWarning: The default value of numeric_only in DataFrame.max is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  max=df.max(axis=0)
C:\Users\abhis\AppData\Local\Temp\ipykernel_28016\4219654605.py:9: FutureWarning: The default value of numeric_only in DataFrame.min is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is de

array([1, 0, 0, ..., 0, 0, 1], dtype=int64)

In [2]:
def prior_prob(y,a):
    count_1=0
    count_0=0
    for _ in range(len(y)):
        if(y[_]==0):
            count_0+=1
        else:
            count_1+=1
    if(a==1):
        return count_1/(count_1+count_0)
    if(a==0):
        return count_0/(count_1+count_0)

def internal_prob(x_tr,x_tep,y_tr,alpha,smooth):
    count_pos=0
    count_neg=0
    pc=0
    nc=0
    #y_tr=y_tr.T
    for i in range(len(y_tr)):
        if y_tr[i] == 1:
            if x_tr[i]== x_tep:
                count_pos +=1
            pc += 1
        
        else:
            if x_tr[i]== x_tep:
                count_neg +=1
            nc+=1
    pos = count_pos/pc
    neg = count_neg/nc
    if smooth=="L":
        if pos==0:
            pos = (count_pos+alpha)/(pc+alpha*14)
        if neg ==0:
            neg = (count_neg+alpha)/(nc+alpha*14)
    if smooth== "N":
        if pos==0:
            pos = 1e-6
        if neg ==0:
            neg== 1e-6

    return pos,neg

def naive_bayes(x_tr,y_tr,x_te,alpha,smooth):
    pos=0
    neg=0
    pos_prob=1
    neg_prob=1
    x_tr=x_tr.T
    for j in range(14):
            pos,neg=internal_prob(x_tr[j],x_te[j],y_tr,alpha,smooth)
            pos_prob*= pos
            neg_prob *= neg
    prior_pos= prior_prob(y_tr,1)
    prior_neg = prior_prob(y_tr,0)

    prob_1= prior_pos*pos_prob
    prob_0 = prior_neg*neg_prob

    if prob_1>prob_0:
        pred=1
    else:
        pred=0
    
    return pred


In [3]:
def evaluate(x_tr,y_tr,x_te,y_te,alpha,smooth):
    count=np.zeros((2,2))
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    
    for i in range(len(x_te)):
        pred = naive_bayes(x_tr,y_tr,x_te[i],alpha,smooth)
        if y_te[i] == 0 and pred == 0:
          tn += 1
        if y_te[i] == 0 and pred == 1:
          fp += 1
        if y_te[i] == 1 and pred == 0:
          fn += 1
        if y_te[i] == 1 and pred == 1:
          tp += 1
    
    count[0][0]=tn
    count[0][1]=fp
    count[1][0]=fn
    count[1][1]=tp

     
    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    F1_score = 2* (precision*recall)/(precision+recall)
    return precision,recall,accuracy,F1_score,count

# p,r,a,f1,arr = evaluate(x_train,y_train,x_test,y_test,1)



In [4]:
def models(smooth):
      df_m = df.sample(frac=2/3)
      x_train= df_m.drop("income",axis=1)
      x_train=df_m
      y_train= df_m["income"]
      x_test= df.drop(df_m.index)
      y_test= x_test["income"]
      x_test.drop("income",axis=1,inplace=True)
      x_train= x_train.to_numpy()
      y_train= y_train.to_numpy()
      x_test=x_test.to_numpy()
      y_test=y_test.to_numpy()

      p,r,a,f1,arr = evaluate(x_train,y_train,x_test,y_test,1,smooth)
      
      print("accuracy :", a*100)
      print(arr)


      return p,r,a,f1

In [5]:
l=['L','N']
for j in l:
  acc=0
  pre=0
  re=0
  f=0

  for i in range(10):
    p,r,a,f1= models(j)
    acc+=a*10
    re+=r*10
    pre+=p*10
    f+=f1*10
  print("avg accuracy = ", acc, j)
  print("avg recall = ", re, j)
  print("avg precision = ", pre, j)
  print("avg f1 score = ", f, j)


accuracy : 82.50414593698176
[[6938. 1309.]
 [ 590. 2017.]]
accuracy : 81.43541551501751
[[6885. 1392.]
 [ 623. 1954.]]
accuracy : 81.33407038879675
[[6835. 1401.]
 [ 625. 1993.]]
accuracy : 81.59203980099502
[[6810. 1415.]
 [ 583. 2046.]]
accuracy : 82.19089736502671
[[6920. 1311.]
 [ 622. 2001.]]
accuracy : 82.91873963515755
[[6984. 1223.]
 [ 631. 2016.]]
accuracy : 81.99742030587802
[[6907. 1329.]
 [ 625. 1993.]]
accuracy : 82.91873963515755
[[6936. 1223.]
 [ 631. 2064.]]
accuracy : 81.38934954855353
[[6839. 1449.]
 [ 571. 1995.]]
accuracy : 81.00239543025613
[[6798. 1403.]
 [ 659. 1994.]]
avg accuracy =  81.92832135618205 L
avg recall =  76.52134425531183 L
avg precision =  59.89492182196532 L
avg f1 score =  67.1815642336943 L
accuracy : 81.58282660770223
[[6783. 1383.]
 [ 616. 2072.]]
accuracy : 81.47226828818869
[[6852. 1386.]
 [ 625. 1991.]]
accuracy : 82.50414593698176
[[6929. 1265.]
 [ 634. 2026.]]
accuracy : 81.30643080891838
[[6804. 1405.]
 [ 624. 2021.]]
accuracy : 82.3198